In [ ]:
## Imports
import os
import numpy as np
import pandas as pd
import nibabel as nb
import warnings
warnings.filterwarnings('ignore')

# Define parameters
subjects = ['sub-001', 'sub-002', 'sub-003', 'sub-004',
            'sub-005', 'sub-006', 'sub-007', 'sub-008']
rois = ['V1', 'V2', 'V3', 'V3AB', 'hMT+', 'LO',
        'VO', 'iIPS', 'sIPS', 'iPCS', 'sPCS', 'mPCS']
gaze_tasks = ['GazeCenter','GazeLeft','GazeRight']
attend_tasks = ['AttendFix', 'AttendBar']
# Define folders
base_dir = '/home/mszinte/disks/meso_S/data/gaze_prf'
bids_dir = "{}".format(base_dir)
pp_dir = "{}/derivatives/pp_data".format(base_dir)

### Compute averaged decoding position and height over runs 

In [ ]:
# compute for each session
for subject in subjects:
    tsv_dir = '{}/{}/decode/tsv'.format(pp_dir, subject)
    try: os.makedirs(tsv_dir)
    except: pass

    for gaze_task in gaze_tasks:
        for attend_task in attend_tasks:
            for session in ['ses-01','ses-02']:
                for roi_num, roi in enumerate(rois):
                
                    # load files
                    ses_fn = "{}/{}/decode/fit/{}_{}_task-{}{}_roi-{}_nvoxels-250_resizefactor-3_mcmcbarpars.tsv".format(pp_dir, subject, subject, session, gaze_task, attend_task, roi)
                    ses_df = pd.read_csv(ses_fn,sep="\t")
                    tr = ses_df.groupby(['chain','time']).time.mean()
                
                    # create dataframe
                    df_roi = pd.DataFrame({'subject': [subject] * tr.shape[0],
                                           'roi': [roi] * tr.shape[0],
                                           'session': [session] * tr.shape[0],
                                           'TR': tr,
                                           'chain': ses_df.groupby(['chain','time']).chain.mean(),
                                           'decoded_x_mean': ses_df.groupby(['chain','time']).x.mean(),
                                           'decoded_x_std': ses_df.groupby(['chain','time']).x.std(),
                                           'decoded_h_mean': ses_df.groupby(['chain','time']).height.mean(),
                                           'decoded_h_std': ses_df.groupby(['chain','time']).height.std()})

                    # across roi
                    if roi_num > 0: df = pd.concat([df,df_roi], ignore_index=True)
                    else: df = df_roi
                
                # Save dataframe as tsv
                df_fn = "{}/{}_{}_task-{}{}_decode_par.pkl".format(tsv_dir,subject,session,gaze_task,attend_task)
                print('saving {}'.format(df_fn))
                df.to_pickle(df_fn)
                
            del df

In [ ]:
# compute over sessions
for subject in subjects:
    tsv_dir = '{}/{}/decode/tsv'.format(pp_dir, subject)
    try: os.makedirs(tsv_dir)
    except: pass

    for gaze_task in gaze_tasks:
        for attend_task in attend_tasks:
            for roi_num, roi in enumerate(rois):

                # load files
                run1_fn = "{}/{}/decode/fit/{}_ses-01_task-{}{}_roi-{}_nvoxels-250_resizefactor-3_mcmcbarpars.tsv".format(pp_dir, subject, subject, gaze_task, attend_task, roi)
                run2_fn = "{}/{}/decode/fit/{}_ses-02_task-{}{}_roi-{}_nvoxels-250_resizefactor-3_mcmcbarpars.tsv".format(pp_dir, subject, subject, gaze_task, attend_task, roi)

                run1_df = pd.read_csv(run1_fn,sep="\t")
                run2_df = pd.read_csv(run2_fn,sep="\t")

                tr = np.nanmean((run1_df.groupby(['time']).time.mean(),run2_df.groupby(['time']).time.mean()),axis=0)
                
                # create dataframe
                df_roi = pd.DataFrame({'subject': [subject] * tr.shape[0],
                                       'roi': [roi] * tr.shape[0],
                                       'TR': tr,
                                       'decoded_x_mean': np.nanmean((run1_df.groupby(['time']).x.mean(),run2_df.groupby(['time']).x.mean()),axis=0),
                                       'decoded_x_std': np.nanmean((run1_df.groupby(['time']).x.std(),run2_df.groupby(['time']).x.std()),axis=0),
                                       'decoded_h_mean': np.nanmean((run1_df.groupby(['time']).height.mean(),run2_df.groupby(['time']).height.mean()),axis=0),
                                       'decoded_h_std': np.nanmean((run1_df.groupby(['time']).height.std(),run2_df.groupby(['time']).height.std()),axis=0)})
                                       
                # across roi
                if roi_num > 0: df = pd.concat([df,df_roi], ignore_index=True)
                else: df = df_roi
                
            # Save dataframe as tsv
            df_fn = "{}/{}_task-{}{}_decode_par.pkl".format(tsv_dir,subject,gaze_task,attend_task)
            print('saving {}'.format(df_fn))
            df.to_pickle(df_fn)
            
            del df

In [ ]:
# compute over bar pass (flipped)
idx = [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 
        35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54,
       ]
for subject in subjects:
    tsv_dir = '{}/{}/decode/tsv'.format(pp_dir, subject)
    try: os.makedirs(tsv_dir)
    except: pass

    for gaze_task in gaze_tasks:
        for attend_task in attend_tasks:
            for roi_num, roi in enumerate(rois):

                # load files
                run1_fn = "{}/{}/decode/fit/{}_ses-01_task-{}{}_roi-{}_nvoxels-250_resizefactor-3_mcmcbarpars.tsv".format(pp_dir, subject, subject, gaze_task, attend_task, roi)
                run2_fn = "{}/{}/decode/fit/{}_ses-02_task-{}{}_roi-{}_nvoxels-250_resizefactor-3_mcmcbarpars.tsv".format(pp_dir, subject, subject, gaze_task, attend_task, roi)

                run1_df = pd.read_csv(run1_fn,sep="\t")
                run2_df = pd.read_csv(run2_fn,sep="\t")
                
                time = np.linspace(0,1,18)
                
                x_mean_run1_flip = np.array([list(run1_df.groupby(['time']).x.mean())[i] for i in idx])
                x_mean_run2_flip = np.array([list(run2_df.groupby(['time']).x.mean())[i] for i in idx])
                x_mean_runs_flip = [x_mean_run1_flip[0:18],x_mean_run1_flip[18:18*2],x_mean_run1_flip[18*2:18*3],x_mean_run1_flip[18*3:18*4],
                                    x_mean_run2_flip[0:18],x_mean_run2_flip[18:18*2],x_mean_run2_flip[18*2:18*3],x_mean_run2_flip[18*3:18*4]]
                                
                h_mean_run1_flip = np.array([list(run1_df.groupby(['time']).height.mean())[i] for i in idx])
                h_mean_run2_flip = np.array([list(run2_df.groupby(['time']).height.mean())[i] for i in idx])
                h_mean_runs_flip = [h_mean_run1_flip[0:18],h_mean_run1_flip[18:18*2],h_mean_run1_flip[18*2:18*3],h_mean_run1_flip[18*3:18*4],
                                    h_mean_run2_flip[0:18],h_mean_run2_flip[18:18*2],h_mean_run2_flip[18*2:18*3],h_mean_run2_flip[18*3:18*4]]
                
                
                # create dataframe
                df_roi = pd.DataFrame({'subject': [subject] * time.shape[0],
                                       'roi': [roi] * time.shape[0],
                                       'Time': time,
                                       'decoded_x_mean_barpass': np.nanmean(x_mean_runs_flip,axis=0),
                                       'decoded_x_std_barpass': np.nanstd(x_mean_runs_flip,axis=0),
                                       'decoded_h_mean_barpass': np.nanmean(h_mean_runs_flip,axis=0),
                                       'decoded_h_std_barpass': np.nanstd(h_mean_runs_flip,axis=0)
                                      })
                # across roi
                if roi_num > 0: df = pd.concat([df,df_roi], ignore_index=True)
                else: df = df_roi
                
            # Save dataframe as pkl
            df_fn = "{}/{}_task-{}{}_decode_par_barpass.pkl".format(tsv_dir,subject,gaze_task,attend_task)
            print('saving {}'.format(df_fn))
            df.to_pickle(df_fn)
            
            del df